In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f'Data successfully loaded with {df.shape[0]} rows and {df.shape[1]} features')
    return df

train_set = load_data('../data/processed/train_processed_v1.csv')
dev_set = load_data('../data/processed/dev_processed_v1.csv')
test_set = load_data('../data/processed/test_processed_v1.csv')

Data successfully loaded with 4751 rows and 19 features
Data successfully loaded with 1356 rows and 19 features
Data successfully loaded with 910 rows and 19 features


In [3]:
def split_feature_target(df, target_variable: str = 'selling_price_log'):
    y = df[target_variable]
    x = df.drop(columns=[target_variable])
    print(f'target_shape: {y.shape} - feature_shape: {x.shape}')
    return x, y

x_train, y_train = split_feature_target(train_set)
x_dev, y_dev = split_feature_target(dev_set)
x_test, y_test = split_feature_target(test_set)

target_shape: (4751,) - feature_shape: (4751, 18)
target_shape: (1356,) - feature_shape: (1356, 18)
target_shape: (910,) - feature_shape: (910, 18)


In [4]:
# import libraries
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import optuna
from sklearn.metrics import root_mean_squared_error, r2_score
import time

In [5]:
def format_duration(seconds: float) -> str:
    if seconds < 60:
        return f"{seconds:.2f}s"
    elif seconds < 3600:
        minutes = int(seconds // 60)
        secs = seconds % 60
        return f"{minutes}m {secs:.2f}s"
    else:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours}h {minutes}m {secs:.2f}s"

In [6]:
# baseline model training
baseline_models = {
    'Ridge' : Ridge(alpha=0.3, max_iter=2000, random_state=1),
    'XGBoost' : XGBRegressor(random_state=42, n_estimators=200, max_depth=3),
    'LightGBM' : LGBMRegressor(max_depth=5, learning_rate=0.3, n_estimators=200, random_state=42),
    'RandomForest' : RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
}

In [7]:
results = {}
print("="*50)
print(f"BASELINE TRAINING INITIALIZED")
print("="*50)
for name, model in baseline_models.items():
    print(f'Training {name} model...')
    start_time = time.time()
    model.fit(x_train, y_train)
    time_elapsed = time.time() - start_time

    preds_train = model.predict(x_train)

    results[name] = {
        "Time_elapsed" : format_duration(time_elapsed),
        "R^2_score" : round(r2_score(y_train, preds_train),4),
        "RMSE" : round(root_mean_squared_error(y_train, preds_train))
    }

for name, result in results.items():
        print(f"{name}")
        print('-'*20)
        print("Time elapsed: ",result['Time_elapsed'])
        print("R^2_Score: ", result['R^2_score'])
        print("RMSE: ",result['RMSE'])
        print ("="*30)

print(f'BASELINE MODEL TRAINING COMPLETED')

BASELINE TRAINING INITIALIZED
Training Ridge model...
Training XGBoost model...
Training LightGBM model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001833 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 870
[LightGBM] [Info] Number of data points in the train set: 4751, number of used features: 18
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [8]:
results = {}
print("="*50)
print(f"CROSS VALIDATION INITIALIZED")
print(f"="*50)

for name, model in baseline_models.items():
    start_time = time.time()
    cv_scores = cross_val_score(model, x_train, y_train, cv=KFold(5), scoring='neg_root_mean_squared_error', n_jobs=-1)
    time_elapsed = time.time() - start_time

    abs_cv_score = abs(cv_scores)
    
    results[name] = {
        'cv_scores_across_folds' : abs_cv_score,
        'cv_score_mean' : round(abs_cv_score.mean(), 4),
        'cv_score_std' : round(abs_cv_score.std(), 4),
        'Time_elapsed' : format_duration(time_elapsed)
    }

for name, result in results.items():
    print(f'Model_name: {name}')
    print(f'='*40)
    print(f'CV Scores: ',result['cv_scores_across_folds'])
    print(f'CV scores_mean: ',result['cv_score_mean'])
    print(f'CV score_std: +/-',result['cv_score_std'])
    print(f'Time_elapsed: ',result['Time_elapsed'])
    print(f'='*40)

print(f'CROSS-VALIDATION MODEL TRAINING COMPLETED')

CROSS VALIDATION INITIALIZED
Model_name: Ridge
CV Scores:  [0.41716765 0.3952506  0.39828129 0.39223018 0.40086055]
CV scores_mean:  0.4008
CV score_std: +/- 0.0087
Time_elapsed:  6.38s
Model_name: XGBoost
CV Scores:  [0.29129551 0.30042265 0.2986982  0.27874239 0.29366269]
CV scores_mean:  0.2926
CV score_std: +/- 0.0077
Time_elapsed:  4.92s
Model_name: LightGBM
CV Scores:  [0.28973033 0.30676242 0.3128978  0.29391002 0.30420172]
CV scores_mean:  0.3015
CV score_std: +/- 0.0085
Time_elapsed:  5.43s
Model_name: RandomForest
CV Scores:  [0.44352197 0.41886004 0.40985105 0.42159388 0.41798678]
CV scores_mean:  0.4224
CV score_std: +/- 0.0113
Time_elapsed:  1.81s
CROSS-VALIDATION MODEL TRAINING COMPLETED


In [9]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [10]:
def objective(trial):
            n_estimators = trial.suggest_int('n_estimators', 100,1000)
            max_depth = trial.suggest_int('max_depth', 3, 15)
            learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
            min_child_weight = trial.suggest_int('min_child_weight', 1 , 10)

            xgbr = XGBRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                min_child_weight=min_child_weight
            )


            scores = cross_val_score(
                    estimator=xgbr,
                    X=x_train, y=y_train,
                    cv=KFold(5),scoring='neg_mean_squared_error',
                    n_jobs=-1
            )

            return -scores.mean()

In [11]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(),pruner=MedianPruner())
study.optimize(objective, n_trials=160)

[I 2026-01-21 15:46:52,415] A new study created in memory with name: no-name-48fc3725-5cf7-4e6f-b9bc-892a981ed8cb
[I 2026-01-21 15:46:55,538] Trial 0 finished with value: 0.10113028937406301 and parameters: {'n_estimators': 410, 'max_depth': 14, 'learning_rate': 0.06038829392804373, 'min_child_weight': 4}. Best is trial 0 with value: 0.10113028937406301.
[I 2026-01-21 15:46:56,774] Trial 1 finished with value: 0.08837976943545418 and parameters: {'n_estimators': 884, 'max_depth': 5, 'learning_rate': 0.10708184112555015, 'min_child_weight': 1}. Best is trial 1 with value: 0.08837976943545418.
[I 2026-01-21 15:46:58,682] Trial 2 finished with value: 0.0866038962032633 and parameters: {'n_estimators': 655, 'max_depth': 7, 'learning_rate': 0.025008013666044538, 'min_child_weight': 1}. Best is trial 2 with value: 0.0866038962032633.
[I 2026-01-21 15:47:02,252] Trial 3 finished with value: 0.10035126443691153 and parameters: {'n_estimators': 670, 'max_depth': 9, 'learning_rate': 0.0965299866

In [15]:
import matplotlib.pyplot as plt
import plotly
optuna.visualization.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
                    111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
                    123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
                    135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
                    147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
                    159],
              'y': [0.10113028937406301, 0.08837976943545418, 0.0866038962032633,
                    0.10035126443691153, 0.08656010798559802, 0.08837981870885267,
                    0.10370125433553223, 0.08758217329344989, 0.11168159039635986,
                    0.2839396111482754, 0.10622604016600334, 0.08679176891964534,
                    0.08929974080510515, 0.09090705534042749, 0.08459212476481498,
                    0.08753893513137004, 0.09146884536603142, 0.10042350863136322,
                    0.08701151729959695, 0.08608189834011143, 0.08553368403970471,
                    0.08709780206248437, 0.08363206449916741, 0.08428077615539901,
                    0.08456517220174968, 0.0841759318257582, 0.08594278058094092,
                    0.08401176785123797, 0.0836082999573127, 0.10060432866720577,
                    0.08759326534455245, 0.08633533645538888, 0.08443012754590523,
                    0.08745082701820608, 0.08466237686885451, 0.09370348214495013,
                    0.08739436812381082, 0.09106323952548691, 0.08328970377231804,
                    0.09422930760538858, 0.09708800202811499, 0.08417897378166266,
                    0.08431302509995645, 0.08559219272730005, 0.08393451801562138,
                    0.08468278129154824, 0.09032401174548085, 0.08533335507203696,
                    0.08909980671997034, 0.08814750967256467, 0.10049662885070165,
                    0.08575704037318158, 0.0842348130305267, 0.08470992962859202,
                    0.09042537604697963, 0.0839031990762163, 0.08609657599829783,
                    0.08365139543957831, 0.08468898175292575, 0.08570142215612121,
                    0.09189579299362469, 0.08594658767035429, 0.08321329292945408,
                    0.08309679748144608, 0.08459575899067322, 0.08381827666624493,
                    0.08409932842968501, 0.1030521625094472, 0.10897561416070078,
                    0.08739783923258478, 0.0836417485918867, 0.08286559213131019,
                    0.08448001615065966, 0.09195310461581628, 0.0837314468808058,
                    0.08878134034207327, 0.08510340059999535, 0.08425690393819489,
                    0.08375038142421135, 0.08389627118224004, 0.08797756354871775,
                    0.08377614740429741, 0.08297683566119857, 0.08288188547922712,
                    0.08482045797375053, 0.08362074694319442, 0.08396099274189792,
                    0.08732040351531895, 0.08455016076550748, 0.08519822592786398,
                    0.08420347248478385, 0.08328098849829646, 0.0843887593201765,
                    0.08446538965141873, 0.0871501781258703, 0.08335265871853151,
                    0.08401660626562071, 0.08523614091282042, 0.10144167818971166,
                    0.0855519801433046, 0.10478467531897447, 0.08325245634320885,
                    0.08368109324341147, 0.08371628610369006, 0.08301064070898072,
                    0.08364241842890921, 0.08331607347620265

In [17]:
optuna.visualization.plot_param_importances(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [min_child_weight (IntDistribution):
                                0.0016964039997593055<extra></extra>, learning_rate
                                (FloatDistribution):
                                0.17509550560863527<extra></extra>, max_depth
                                (IntDistribution):
                                0.24174886864374626<extra></extra>, n_estimators
                                (IntDistribution):
                                0.5814592217478591<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [<0.01, 0.18, 0.24, 0.58],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.0016964039997593055, 0.17509550560863527,
                    0.24174886864374626, 0.5814592217478591],
              'y': [min_child_weight, learning_rate, max_depth, n_estimators]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [ ]:
optuna.visualization.plot_slice(study)